# Task
Решаю задачу бинарной классификации временных рядов.
Предсказываю бар определенного типа:
- для лонга: верхняя тень в несколько раз больше чем нижняя
- для шорта: нижняя тень в несколько раз больше чем верхняя

Классы не сбалансированны, целевой класс не более 10-15% в выборке в зависимости от настройки

Для моих целей задачи классификации смотрю на метрики:
- Precision - точность, т.к. сколько в % от предсказаний класса 1 действительно правда. Хотелось бы увидитеть хотя бы 20% в этом месте я в нуле если соотношение доходность/риск как 4к1.
- Recall - полнота, сколько % объектов класса 1 я нашел относительно всех объектов класса 1. Тут я на самом деле не закладываюсь, просто наблюдаю что бы были адекватные значения. Если всего объектов класс 1 на истории 4т, то 10% словить при высоких значениях Precision это тоже хорошо. А вот 1%, это уже маловато, это всего 40 сделок.
- f1 использую как объединение предыдущих метрик.

# Load Data
Загружаю исторические данные из паркет файлов.

По умолчанию я использую часовик. Но тут в список или лучше в словарь могу загрузить сразу много файлов разных таймфреймов
- 2min
- 3min
- 5min
- 10min
- 15min
- 20min
- 30min
- 1H
- 2H
- 3H
- 4H

И в цикле в итоге искать где лучше всего алгоритм работает

# Base Visualization

# EDA
Исследование данных

Проведение тестов на стационарность

Задача получить стационарные временные ряды, отчистить их от трендов. Что бы тренировочные и тестовые части находились в одном пространстве признаков.

# Create Target Category Feature 
Создаю целевой признак

Это фактически очередные гиперпараметры.

Я могу задавать уровень риска в лог величинах - risk_level

и коэффициент доходность/риск в виде множителя - risk_factor

Т.е. текущий случай это:
* risk_level = 0.0025
* risk_factor = 4


In [ ]:
risk_level = 0.0025
profit_factor = 4
profit_level = risk_level * profit_factor

# Feature Engineering
* Преобразую исходные данные
* Получаю новые признаки. 
* Удаляю ненужные.
* Обогощаю данные.

* Объемы и сделки:
Можно считать средний объем сделки. 
- AvrTrade = Volume / Trades
- Объем в долларах, т.е. нормированный объем VolUSD = Volume * Close
- Объем на единицу диапазона.

# Outliers
Обработка выбросов. Пробовать прорядить выборку удалением записей по методу:
- 3х сигм
- методу Тьюки
- Isolation Forest
- Local Outlier Factor
- Minimun Covariance Determinant

Но обработку надо делать уже после разделения выборки на train/test

# Transform TimeSeries to Dataset for Supervised Learning
Тут определяется глубина последовательности данных которая будет использоваться для построения прогноза T.

Т это так же гиперпараметр, его так же нужно будет искать по сетке.

In [ ]:
T = 24 # данные за сутки беру

### Различия между ML и DeepLearning
Как я понимаю сейчас алгоритмы ML принимают на вход только вектор признаков для конкретного объекта датасета.
Т.е. Close текущего бара, или Log_Return на текущем баре. Но каждый признак состоит из T этих признаков в прошлое.

И получаетмя что это двумерный массив.

Но для подачи данных в ML модели мне эту матрицу нужно развернуть наверное с помощью flatten в вектор.
А вот в модели ANN я могу подавать матрицу размерностью T * D.

Т.е. в функции трасформации и создания датасета из временного ряда должен быть аргумент для чего мы делаем данные.

# Validation Preperation
Первое разделение данных на общую (тренировочную/валидационную) и тестовую.
Тут разделение нужно произовить без перемешивания и стратификации.

Это будет похоже на релаьную симуляцию работы. Когда общая левая часть это все доступные текущие данные. А левая это то что будет появляться в режиме реального времени. И на чем будет производиться тестирование модели, а фактически реальное использование модели для торговли.


Но может я не прав и деление тут тоже надо производить случайным образом и со стратификацией. Теперь кажется что так тестирование будет более полным и равномерным.

# Split Data
Теперь левую общую часть надо разделить на тествоую и валидационную случайным образом и с учетом несбалансированности классов, применяя стратификацию.

# Data Normalization
Теперь наверное можно нормальзовать данные.
Скалер обучаю только на X_train, а трансформирую все три матрицы X_test, X_valid, X_test